In [1]:
import pandas as pd
import seaborn as sns
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
from datetime import date

In [2]:
# Load Train Data
rawdata = pd.read_csv('../data/raw/train_aggr.csv',sep=';')

In [3]:
# Load Submit Data
submittion = pd.read_csv('../data/raw/ejemploRespuesta.csv')

In [4]:
submittion.columns = ['id_pos','unidades_pred']

In [5]:
rawdata.head()

,fecha_venta,id_pos,unidades,canal,competidores,ingreso_mediana,ingreso_promedio,densidad_poblacional,pct_0a5,pct_5a9,...,pct_bachelors,pct_doctorados,pct_secundario,pct_master,pct_bicicleta,pct_omnibus,pct_subtes,pct_taxi,pct_caminata,mediana_valor_hogar
0,2012-10-01,8495,4,MAXIKIOSCO,14.0,49367.31,65176.325,7981.955,6.695,6.26,...,16.75,0.61,32.42,5.35,0.18,1.16,0.0,0.145,3.01,152962.485
1,2012-11-10,8495,1,MAXIKIOSCO,14.0,49367.31,65176.325,7981.955,6.695,6.26,...,16.75,0.61,32.42,5.35,0.18,1.16,0.0,0.145,3.01,152962.485
2,2012-12-03,8495,1,MAXIKIOSCO,14.0,49367.31,65176.325,7981.955,6.695,6.26,...,16.75,0.61,32.42,5.35,0.18,1.16,0.0,0.145,3.01,152962.485
3,2012-12-13,8495,1,MAXIKIOSCO,14.0,49367.31,65176.325,7981.955,6.695,6.26,...,16.75,0.61,32.42,5.35,0.18,1.16,0.0,0.145,3.01,152962.485
4,2012-12-14,8495,2,MAXIKIOSCO,14.0,49367.31,65176.325,7981.955,6.695,6.26,...,16.75,0.61,32.42,5.35,0.18,1.16,0.0,0.145,3.01,152962.485


In [6]:
rawdata.shape

(256913, 36)

In [7]:
rawdata['fecha_venta'] = pd.to_datetime(rawdata['fecha_venta'])

In [8]:
rawdata['fecha_venta_norm'] = rawdata['fecha_venta'].apply(lambda x : date(x.year,x.month,1))

In [9]:
rawdata.fecha_venta_norm.unique()

array([datetime.date(2012, 10, 1), datetime.date(2012, 11, 1),
       datetime.date(2012, 12, 1), datetime.date(2013, 8, 1),
       datetime.date(2013, 11, 1), datetime.date(2013, 12, 1),
       datetime.date(2014, 11, 1), datetime.date(2014, 10, 1),
       datetime.date(2013, 10, 1), datetime.date(2014, 9, 1),
       datetime.date(2013, 9, 1), datetime.date(2013, 6, 1),
       datetime.date(2013, 7, 1)], dtype=object)

In [10]:
# Filtramos los meses que consideramos buenos para el entrenamiento (11 y 12)
rawdata = rawdata[rawdata.fecha_venta_norm.isin([date(2012, 11, 1),date(2012, 12, 1),
                                                date(2013, 11, 1), date(2013, 12, 1), date(2014, 11, 1)])]

In [11]:
rawdata.shape

(256110, 37)

In [43]:
predictors = ['unidades', 'canal', 'competidores',
       'ingreso_mediana', 'densidad_poblacional',
       'pct_0a5', 'pct_5a9', 'pct_10a14', 'pct_15a19', 'pct_20a24',
       'pct_25a29', 'pct_30a34', 'pct_35a39', 'pct_40a44', 'pct_45a49',
       'pct_50a54', 'pct_55a59', 'pct_60a64', 'pct_65a69', 'pct_70a74',
       'pct_75a79', 'pct_80a84', 'pct_85ainf', 'pct_bachelors',
       'pct_doctorados', 'pct_secundario', 'pct_master', 'pct_bicicleta',
       'pct_omnibus', 'pct_subtes', 'pct_taxi', 'pct_caminata',
       'mediana_valor_hogar']

In [42]:
rawdata.columns

Index(['fecha_venta', 'id_pos', 'unidades', 'canal', 'competidores',
       'ingreso_mediana', 'ingreso_promedio', 'densidad_poblacional',
       'pct_0a5', 'pct_5a9', 'pct_10a14', 'pct_15a19', 'pct_20a24',
       'pct_25a29', 'pct_30a34', 'pct_35a39', 'pct_40a44', 'pct_45a49',
       'pct_50a54', 'pct_55a59', 'pct_60a64', 'pct_65a69', 'pct_70a74',
       'pct_75a79', 'pct_80a84', 'pct_85ainf', 'pct_bachelors',
       'pct_doctorados', 'pct_secundario', 'pct_master', 'pct_bicicleta',
       'pct_omnibus', 'pct_subtes', 'pct_taxi', 'pct_caminata',
       'mediana_valor_hogar', 'fecha_venta_norm'],
      dtype='object')

In [52]:
X = rawdata[predictors].copy()

In [53]:
y = rawdata.unidades

In [54]:
X.drop('unidades', inplace=True, axis=1)

In [55]:
# fill Nans
for c in X.columns[X.dtypes != 'object']:
    X[c].fillna(X[c].median(), inplace=True)

In [58]:
# Label encoder
for c in X.columns[X.dtypes == 'object']:
    X[c] = X[c].factorize()[0]

In [63]:
X = sm.add_constant(X)

/opt/anaconda3/envs/signalenv/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [68]:
model = sm.OLS(y, X).fit()
predictions = model.predict(X) # make the predictions by the model

# Print out the statistics
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               unidades   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.376
Date:                Sun, 16 Jun 2019   Prob (F-statistic):           6.15e-27
Time:                        22:08:44   Log-Likelihood:            -5.5365e+05
No. Observations:              239834   AIC:                         1.107e+06
Df Residuals:                  239801   BIC:                         1.108e+06
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -1.6365      3.891     -0.421      0.674      -9.263       5.990
canal                    0.8709      0.077     11.249      0.000       0.719       1.023
competidores            -0.0001      0.000     -0.991      0.322      -0.000       0.000
ingreso_mediana      -1.094e-06   8.36e-07     -1.307      0.191   -2.73e-06    5.46e-07
densidad_poblacional -3.077e-07   7.98e-07     -0.386      0.700   -1.87e-06    1.26e-06
pct_0a5                  0.0329      0.039      0.834      0.404      -0.044       0.110
pct_5a9                  0.0094      0.042      0.222      0.824      -0.074       0.092
pct_10a14                0.0347      0.039      0.879      0.380      -0.043       0.112
pct_15a19                0.0531      0.039      1.357      0.175      -0.024       0.130
pct_20a24                0.0196      0.039      0.502      0.616      -0.057       0.096
pct_25a29                0.0402      0.039      1.022      0.307      -0.037       0.117
pct_30a34                0.0293      0.040      0.726      0.468      -0.050       0.108
pct_35a39                0.0266      0.041      0.653      0.514      -0.053       0.107
pct_40a44                0.0253      0.041      0.621      0.534      -0.055       0.105
pct_45a49                0.0318      0.041      0.777      0.437      -0.048       0.112
pct_50a54                0.0446      0.041      1.099      0.272      -0.035       0.124
pct_55a59                0.0193      0.040      0.477      0.634      -0.060       0.099
pct_60a64                0.0372      0.041      0.919      0.358      -0.042       0.117
pct_65a69                0.0070      0.041      0.172      0.863      -0.073       0.087
pct_70a74                0.0306      0.041      0.738      0.461      -0.051       0.112
pct_75a79                0.0665      0.043      1.550      0.121      -0.018       0.151
pct_80a84                0.0460      0.044      1.055      0.292      -0.040       0.132
pct_85ainf               0.0086      0.040      0.216      0.829      -0.070       0.087
pct_bachelors            0.0027      0.002      1.264      0.206      -0.001       0.007
pct_doctorados          -0.0001      0.009     -0.014      0.989      -0.017       0.017
pct_secundario           0.0036      0.002      2.313      0.021       0.001       0.007
pct_master               0.0054      0.004      1.347      0.178      -0.002       0.013
pct_bicicleta            0.0215      0.007      3.001      0.003       0.007       0.036
pct_omnibus             -0.0002      0.002     -0.071      0.943      -0.005       0.004
pct_subtes              -0.0007      0.002     -0.333      0.739      -0.005       0.003
pct_taxi                 0.0101      0.021      0.488      0.626      -0.031       0.051
pct_caminata            -0.0043      0.002     -2.209    

In [69]:
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:               unidades   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     6.376
Date:                Sun, 16 Jun 2019   Prob (F-statistic):           6.15e-27
Time:                        22:14:53   Log-Likelihood:            -5.5365e+05
No. Observations:              239834   AIC:                         1.107e+06
Df Residuals:                  239801   BIC:                         1.108e+06
Df Model:                          32                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                   -1.6365 

#### Linear Regression

In [86]:
X.drop('const', inplace=True, axis=1)

In [87]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [90]:
model = LinearRegression(n_jobs=4).fit(X_train, y_train)


In [91]:
r_sq = model.score(X_train, y_train)
print('coefficient of determination:', r_sq)

print('intercept:', model.intercept_)

print('slope:', model.coef_)

coefficient of determination: 0.0007882923900750516
intercept: -0.6682183467606695
slope: [ 8.13306557e-01 -2.30496605e-04 -1.32644614e-06  2.64565956e-08
  2.42921357e-02  1.62953630e-03  2.46772653e-02  4.29687689e-02
  1.01042408e-02  3.36231126e-02  2.08981272e-02  1.20765467e-02
  2.07250073e-02  1.67095208e-02  4.13316258e-02  1.32831810e-02
  1.90784534e-02 -3.00023022e-03  3.56842973e-02  4.84769912e-02
  2.80202157e-02  6.09532067e-03  2.64568994e-03 -2.88928835e-03
  3.31144646e-03  8.39960898e-03  1.97588329e-02 -3.53885648e-03
 -1.42670537e-03 -9.01250578e-03 -2.70386610e-03  5.12047905e-08]


In [98]:
y_pred = model.predict(X_test)

In [94]:
from sklearn.metrics import mean_absolute_error

In [78]:
from sklearn.metrics import mean_squared_error

In [99]:
mean_absolute_error(y_test, y_pred)

1.571959460235229

In [102]:
mean_squared_error(y_test, y_pred)

5.971510646796166

### Train Final Model

In [103]:
model = LinearRegression(n_jobs=4).fit(X, y)

In [104]:
r_sq = model.score(X_train, y_train)
print('coefficient of determination:', r_sq)

print('intercept:', model.intercept_)

print('slope:', model.coef_)

coefficient of determination: 0.0007475499909412875
intercept: -1.6365175382024901
slope: [ 8.70905905e-01 -1.12537594e-04 -1.09354694e-06 -3.07725037e-07
  3.28578227e-02  9.40190156e-03  3.46916655e-02  5.30874599e-02
  1.95930534e-02  4.02258919e-02  2.93068865e-02  2.66377692e-02
  2.53146258e-02  3.17748793e-02  4.45670177e-02  1.92741333e-02
  3.72199997e-02  6.98816004e-03  3.06150826e-02  6.65113521e-02
  4.60427094e-02  8.63861108e-03  2.67869965e-03 -1.20443590e-04
  3.58852659e-03  5.39232994e-03  2.14770958e-02 -1.63395871e-04
 -6.64356302e-04  1.01362499e-02 -4.33665929e-03  3.92879130e-08]


In [105]:
y_pred = model.predict(X)

In [106]:
mean_absolute_error(y, y_pred)

1.574054994556819